# CS 101 Final Project

## Part 1: Exploratory Data Analysis

### Dataset Exploration

In [30]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score, f1_score
from tqdm import tqdm
import json
import re
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import re
import json

In [31]:
seed = 42

In [32]:
data_er = pd.read_csv("https://raw.githubusercontent.com/behavioral-data/Empathy-Mental-Health/refs/heads/master/dataset/emotional-reactions-reddit.csv", index_col=0)
data_exp = pd.read_csv("https://raw.githubusercontent.com/behavioral-data/Empathy-Mental-Health/refs/heads/master/dataset/explorations-reddit.csv", index_col=0)
data_int = pd.read_csv("https://raw.githubusercontent.com/behavioral-data/Empathy-Mental-Health/refs/heads/master/dataset/interpretations-reddit.csv", index_col=0)
sampled_data = data_exp.sample(n=1000, random_state=seed)

In [33]:
#  keep the data that we are using for each model constant
sampled_data.to_csv('sampled_data.csv', index=False)

In [34]:
few_shot_data = data_exp.groupby('level').apply(lambda x: x.sample(1)).reset_index(drop=True)

/var/folders/g3/6zxrpl516gj47w81m5p9v8rc0000gn/T/ipykernel_18804/4205323108.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  few_shot_data = data_exp.groupby('level').apply(lambda x: x.sample(1)).reset_index(drop=True)


## Part 3: Using Large Language Models

#### API Key

In [ ]:
API_KEY = ""

#### Helper Functions

In [36]:
for i in range(len(few_shot_data)):
  print([few_shot_data['seeker_post'].iloc[i], sampled_data['response_post'].iloc[i]])

["I hate myself so much that I'd rather not commit suicide just to punish myself. I'm dead on the inside.", 'You piece of shit how Dare you make such an insensitive comment to someone that is potentially suicidal FUCK YOU!!! Have some empathy you waste of a human being.']
["I'm ready. It's my own fault. I'm ready to go.", 'When I watch films.Unable to concentrate on anything else,almost unable to take rest.']
['Self destructive after a night out. How come im at my most fuck everything mood at the end of a night out??', "I'm in my 50's and have been struggling with depression since I was a child. In my experience, the teen years were quite difficult, so I would say yes, that it did get better. Until it got worse again. Then better. Then worse. Then better. Many many wonderful things have happened during the good times. And I've survived 100% of my bad times. What I mostly have learned over these decades of illness, is that things always shift, eventually. I rely on docs, meds, therapy, 

In [37]:
def save_response_data(prompt_name, ai_answers, rationales=None, model_name='gpt-4o'):
  sampled_data = pd.read_csv('sampled_data.csv')
  sampled_data['level'] = ai_answers
  if rationales:
    sampled_data['rationales'] = rationales
  sampled_data.to_csv(f'coded_sampled_data_{model_name}_{prompt_name}.csv', index=False)

In [38]:
def calculate_f1_score_debated(ai_csv_path,  ground_truth_csv_path='sampled_data.csv'):
  ai_answers = pd.read_csv(ai_csv_path)['debated_ai_level']
  levels = pd.read_csv(ground_truth_csv_path)['level']
  f1 = f1_score(levels, ai_answers, average='macro')
  print(f"F1 Score (Debated AI): {f1:.2f}")

def calculate_f1_score_initial(csv_path, ground_truth_csv_path='sampled_data.csv'):
  ai_answers = pd.read_csv(csv_path)['initial_ai_level']
  levels = pd.read_csv(ground_truth_csv_path)['level']
  f1 = f1_score(levels, ai_answers, average='macro')
  print(f"F1 Score (Initial AI): {f1:.2f}")

def calculate_f1_score_verifier(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['verifier_suggested_level']
    levels = pd.read_csv(ground_truth_csv_path)['level']
    f1 = f1_score(levels, ai_answers, average='macro')
    print(f"F1 Score (Verifier Suggestion): {f1:.2f}")

def calculate_f1_score_roles(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['roles_based_level']
    levels = pd.read_csv(ground_truth_csv_path)['level']
    f1 = f1_score(levels, ai_answers, average='macro')
    print(f"F1 Score (Verifier Suggestion): {f1:.2f}")

In [ ]:
def calculate_f1_score_debated(ai_csv_path,  ground_truth_csv_path='sampled_data.csv'):
  ai_answers = pd.read_csv(ai_csv_path)['debated_ai_level']
  levels = pd.read_csv(ground_truth_csv_path)['level']
  f1 = f1_score(levels, ai_answers, average='macro')
  print(f"F1 Score (Debated AI): {f1:.2f}")

def calculate_f1_score_initial(csv_path, ground_truth_csv_path='sampled_data.csv'):
  ai_answers = pd.read_csv(csv_path)['initial_ai_level']
  levels = pd.read_csv(ground_truth_csv_path)['level']
  f1 = f1_score(levels, ai_answers, average='macro')
  print(f"F1 Score (Initial AI): {f1:.2f}")

def calculate_f1_score_verifier(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['verifier_suggested_level']
    levels = pd.read_csv(ground_truth_csv_path)['level']
    f1 = f1_score(levels, ai_answers, average='macro')
    print(f"F1 Score (Verifier Suggestion): {f1:.2f}")

def calculate_f1_score_roles(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['roles_based_level']
    levels = pd.read_csv(ground_truth_csv_path)['level']
    f1 = f1_score(levels, ai_answers, average='macro')
    print(f"F1 Score (Verifier Suggestion): {f1:.2f}")

def calculate_cohens_kappa_debated(csv1, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv1)
    df2 = pd.read_csv(ground_truth_csv_path)
    ratings1 = df1['debated_ai_level']
    ratings2 = df2['level']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Debated AI): {normal_kappa:.2f}\nWeighted Cohen's Kappa (Debated AI): {weighted_kappa:.2f}")

def calculate_cohens_kappa_initial(csv_path, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv_path)
    df2 = pd.read_csv(ground_truth_csv_path)
    ratings1 = df1['initial_ai_level']
    ratings2 = df2['level']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Initial AI): {normal_kappa:.2f}\nWeighted Cohen's Kappa (Initial AI): {weighted_kappa:.2f}")

def calculate_cohens_kappa_verifier(csv_path, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv_path)
    df2 = pd.read_csv(ground_truth_csv_path)
    ratings1 = df1['verifier_suggested_level']
    ratings2 = df2['level']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Verifier Suggestion): {normal_kappa:.2f}")
    print(f"Weighted Cohen's Kappa (Verifier Suggestion): {weighted_kappa:.2f}")

def calculate_cohens_kappa_roles(csv_path, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv_path)
    df2 = pd.read_csv(ground_truth_csv_path)
    ratings1 = df1['roles_based_level']
    ratings2 = df2['level']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Verifier Suggestion): {normal_kappa:.2f}")
    print(f"Weighted Cohen's Kappa (Verifier Suggestion): {weighted_kappa:.2f}")

In [39]:
def clean_json(json_str):
  pattern = r'^```json\s*(.*?)\s*```$'
  json_str = re.sub(pattern, r'\1', json_str, flags=re.DOTALL)
  json_str = re.sub(r'("Level": )(\w+)', r'\1"\2"', json_str)
  json_str = json_str.replace("['']", "[]")
  json_str = json_str.replace("[\"\"]", "[]")
  return json_str

In [41]:
def convert_response_data(json_ai):
    json_answers = []
    for i, json_str in enumerate(json_ai):
        try:
            json_str = clean_json(json_str)
            data = json.loads(json_str)
            json_answers.append(data)
        except json.JSONDecodeError as e:
            print(f"Error parsing item {i}: {e}")
            data = {"Level": "C", "Rationale": ""}
            json_answers.append(data)
    ai_levels = [answer_dict[answer['Level']] for answer in json_answers]
    ai_rationales = [answer['Rationale'] for answer in json_answers]
    return ai_levels, ai_rationales

In [42]:
def rationale_similarity(list1):
    """
    Compares two lists of lists of strings and computes a similarity score based on overlapping elements.

    :param list1: First list of lists of strings
    :param list2: Second list of lists of strings
    :return: Similarity score (0 to 1) and overlap details
    """
    ground_truth_rationales = []
    for i in range(response_count):
        ground_truth_rationales.append([] if sampled_data['rationales'].iloc[i] is np.nan else sampled_data['rationales'].iloc[i].split("|")[:-1])

    list2 = ground_truth_rationales
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same number of sublists")

    total_score = 0
    total_possible = 0
    overlap_details = []

    for sub1, sub2 in zip(list1, list2):
        set1, set2 = set(sub1), set(sub2)
        intersection = set1 & set2
        union = set1 | set2

        similarity = len(intersection) / len(union) if union else 1
        total_score += similarity
        total_possible += 1

        overlap_details.append({
            "sublist_1": sub1,
            "sublist_2": sub2,
            "overlap": list(intersection),
            "similarity": similarity
        })

    overall_similarity = total_score / total_possible if total_possible else 1
    return overall_similarity, overlap_details


## Debate and Decide Empathy

In [43]:
def debate_and_decide(seeker_post, response_post, initial_level, original_prompt, client):
    """
    Uses additional agents to debate and decide on the final empathy level.
    """
    # Pro-view debater
    pro_view_prompt = f'''
    Given the following conversation snippet:
    Seeker Post: {seeker_post}
    Response Post: {response_post}

    And an initial classification of the response post as level "{initial_level}" based on this prompt "{original_prompt}".
    '''
    pro_view_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a mental health professional arguing in favor of an empathy classification."},
            {"role": "user", "content": pro_view_prompt}
        ]
    )
    pro_view_argument = pro_view_completion.choices[0].message.content.strip()

    # Anti-view debater
    anti_view_prompt = f'''
    Given the following conversation snippet:
    Seeker Post: {seeker_post}
    Response Post: {response_post}

    And an initial classification of the response post as level "{initial_level}" based on this prompt "{original_prompt}".
    Answer with the following JSON format:
    {{
        "Suggested_Level": "A", "B", or "C",
        "Argument": "Your reasoning here"
    }}
    '''
    anti_view_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a mental health professional arguing against an empathy classification and suggesting an alternative."},
            {"role": "user", "content": anti_view_prompt}
        ]
    )
    anti_view_response_str = anti_view_completion.choices[0].message.content.strip()

    # Attempt to parse the anti-view JSON response
    try:
        anti_view_data = json.loads(clean_json(anti_view_response_str))
        anti_view_suggested_level = anti_view_data.get("Suggested_Level", "C") # Default to C if parsing fails
        anti_view_argument = anti_view_data.get("Argument", "Could not parse argument.")
    except json.JSONDecodeError:
        anti_view_suggested_level = "C" # Default to C if parsing fails
        anti_view_argument = "Error parsing anti-view response."


    # Decider
    decider_prompt = f'''
    Given the following conversation snippet:
    Seeker Post: {seeker_post}
    Response Post: {response_post}

    An initial classification was made as level "{initial_level}" for this prompt "{original_prompt}".

    Here is an argument against that classification, suggesting level "{anti_view_suggested_level}":
    {anti_view_argument}

    Here is an argument supporting that classification:
    {pro_view_argument}


    Considering the original texts and both arguments, decide on the final empathy level (A, B, or C) that best fits the response post based on exploration.
    Answer with the final level (A, B, or C) and a brief rationale.
    Answer with the following JSON format:
    {{
        "Final_Level": "A", "B", or "C",
        "Rationale": "Your reasoning for the final decision"
    }}
    '''
    decider_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a neutral mental health professional evaluating arguments and making a final decision on an empathy classification."},
            {"role": "user", "content": decider_prompt}
        ]
    )
    decider_response_str = decider_completion.choices[0].message.content.strip()

    # Attempt to parse the decider JSON response
    try:
        decider_data = json.loads(clean_json(decider_response_str))
        final_level = decider_data.get("Final_Level", "C") # Default to C if parsing fails
        final_rationale = decider_data.get("Rationale", "Could not parse rationale.")
    except json.JSONDecodeError:
        final_level = "C" # Default to C if parsing fails
        final_rationale = "Error parsing decider response."

    return final_level, final_rationale, pro_view_argument, anti_view_argument, anti_view_suggested_level

In [44]:

def label_empathy(answer_file):
    client = OpenAI(api_key=API_KEY)
    answer_dict = {
        "C": 0,
        "B": 1,
        "A": 2
    }
    
    response_count = len(sampled_data)

    # Define column names for the CSV
    # Start with columns from sampled_data and add new AI-generated columns
    output_columns = list(sampled_data.columns) + [
        'initial_ai_level_char',       # e.g., "A", "B", "C"
        'initial_ai_level_num',        # e.g., 2, 1, 0
        'debated_ai_level_char',       # Final level from debate
        'debated_ai_level_num',
        'debated_ai_rationale',
        'pro_view_argument',
        'anti_view_argument',
        'anti_view_suggested_level_char', # Suggested level by anti-view
        'anti_view_suggested_level_num'
    ]

    # Initialize CSV file with headers (this will overwrite the file if it exists)
    pd.DataFrame(columns=output_columns).to_csv(answer_file, index=False)
    print(f"Initialized {answer_file} with headers.")

    for i in tqdm(range(100)):
        chosen_messages = [sampled_data['seeker_post'].iloc[i], sampled_data['response_post'].iloc[i]]
        
        # Base prompt for initial classification
        initial_classification_prompt = f'''
        You will be provided with a snippet of a conversation.
        The first piece of text will be a support seeking post from an online user, and the second will be a response posted in response to the seeker's post.
        Classify each query based on the level of empathy in the response post in the context of the seeker post.

        We measure the level of empathy in terms of exploration, which is the pursuit of further information to better understand the seeker's situation.

        Levels Of Empathy:
        A. Strong Exploration: The response includes specific questions about the seeker's situation
        B. Weak Exploration: The response includes general questions about the seeker's situation.
        C. No Exploration: The response does not include any questions about the seeker's situation to further understand the seeker's situation.

        Please classify the following responses based on the level of empathy in the response post in the context of the seeker post.
        Seeker Post: {chosen_messages[0]}
        Response Post: {chosen_messages[1]}

        Answer with A, B, C and no justification for your answer.
        Include no punctuation after your answer.
        '''

        current_initial_answer_char = "C"  # Default value
        try:
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "developer", "content": "You are a mental health professional."},
                    {"role": "user", "content": initial_classification_prompt}
                ]
            )
            raw_initial_answer = completion.choices[0].message.content.strip()
            if raw_initial_answer in answer_dict:
                 current_initial_answer_char = raw_initial_answer
            else:
                print(f"Warning: Unexpected initial answer '{raw_initial_answer}' for item {i}. Defaulting to 'C'.")
        except Exception as e:
            print(f"Error getting initial AI level for item {i}: {e}. Defaulting to 'C'.")
            # current_initial_answer_char remains "C"

        # Debate and Decide
        final_level_char = "C" # Default
        final_rationale = "Error in debate"
        pro_arg = "Error in debate"
        anti_arg = "Error in debate"
        anti_level_char = "C" # Default

        try:
            final_level_char, final_rationale, pro_arg, anti_arg, anti_level_char = debate_and_decide(
                chosen_messages[0],
                chosen_messages[1],
                current_initial_answer_char, # Pass the character A, B, or C
                initial_classification_prompt, # Pass the original prompt text used for initial classification
                client
            )
        except Exception as e:
            print(f"Error during debate_and_decide for item {i}: {e}")
            # Defaults will be used

        # Prepare data for the current row
        # Start with original data from sampled_data
        row_data_dict = sampled_data.iloc[i].to_dict()

        # Add all AI generated data
        row_data_dict['initial_ai_level_char'] = current_initial_answer_char
        row_data_dict['initial_ai_level_num'] = answer_dict.get(current_initial_answer_char, 0)
        
        row_data_dict['debated_ai_level_char'] = final_level_char
        row_data_dict['debated_ai_level_num'] = answer_dict.get(final_level_char, 0)
        row_data_dict['debated_ai_rationale'] = final_rationale
        
        row_data_dict['pro_view_argument'] = pro_arg
        row_data_dict['anti_view_argument'] = anti_arg
        
        row_data_dict['anti_view_suggested_level_char'] = anti_level_char
        row_data_dict['anti_view_suggested_level_num'] = answer_dict.get(anti_level_char, 0)

        # Convert the dictionary for the current row to a DataFrame
        # Ensure the columns argument is used to maintain order and handle missing keys gracefully if any
        current_row_df = pd.DataFrame([row_data_dict], columns=output_columns)
        
        # Append to CSV. mode='a' for append, header=False because headers are already written.
        current_row_df.to_csv(answer_file, mode='a', header=False, index=False)

    print(f"Processing complete. All results saved incrementally to {answer_file}")

# ...existing code...
# Replace the existing label_empathy function in cell eafa7076 with the one above.
# The final block of code in your original label_empathy function that starts with:
# output_df = sampled_data.copy()
# ... and ends with output_df.to_csv(...)
# is no longer needed because saving is done incrementally.

In [48]:
def calculate_f1_score(csv_path, column_name, print_name):
    ai_answers = pd.read_csv(csv_path)[column_name]
    levels = pd.read_csv(csv_path)['level']
    f1 = f1_score(levels, ai_answers, average='macro')
    print(f"F1 Score ({print_name}): {f1:.2f}")

def calculate_cohens_kappa(csv_path, column_name, print_name):
    df = pd.read_csv(csv_path)
    ratings1 = df[column_name]
    ratings2 = df['level']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa ({print_name}): {normal_kappa:.2f}\nWeighted Cohen's Kappa ({print_name}): {weighted_kappa:.2f}")

In [45]:
answer_file = 'coded_sampled_data_gpt-4o_no_shot_debated.csv'
label_empathy(answer_file)


Initialized coded_sampled_data_gpt-4o_no_shot_debated.csv with headers.


100%|██████████| 100/100 [14:07<00:00,  8.48s/it]

Processing complete. All results saved incrementally to coded_sampled_data_gpt-4o_no_shot_debated.csv


In [50]:
print("Metrics for Initial AI Response:")
calculate_f1_score(answer_file,'initial_ai_level_num', 'Initial AI Level')
calculate_cohens_kappa(answer_file, 'initial_ai_level_num', 'Initial AI Level')

print("\nMetrics for Debated AI Response:")
calculate_f1_score(answer_file, 'debated_ai_level_num', 'Debated AI Level')
calculate_cohens_kappa(answer_file, 'debated_ai_level_num', 'Debated AI Level')

Metrics for Initial AI Response:
F1 Score (Initial AI Level): 0.63
Cohen's Kappa (Initial AI Level): 0.62
Weighted Cohen's Kappa (Initial AI Level): 0.73

Metrics for Debated AI Response:
F1 Score (Debated AI Level): 0.54
Cohen's Kappa (Debated AI Level): 0.53
Weighted Cohen's Kappa (Debated AI Level): 0.65


## Verifyband Decide Empathy

In [53]:
def verifier_and_decide_empathy(seeker_post, response_post, initial_level, original_prompt, client):
    # Verifier Agent
    verifier_prompt = f"""
    You are a verifier evaluating whether the initial classification of empathy level is justified.

    Classification: '{initial_level}'

    Seeker Post: {seeker_post}
    Response Post: {response_post}

    Based on the EPITOME rubric for Exploration, determine if this label is correct:
    - A: Strong Exploration (asks specific questions)
    - B: Weak Exploration (asks general questions)
    - C: No Exploration (no questions at all)

    Respond in JSON:
    {{
        "Agree": true or false,
        "Suggested_Level": "A", "B", or "C",
        "Justification": "..."
    }}
    """

    verifier_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a mental health researcher verifying empathy annotations."},
            {"role": "user", "content": verifier_prompt}
        ]
    )

    verifier_raw = verifier_completion.choices[0].message.content.strip()
    try:
        verifier_data = json.loads(clean_json(verifier_raw))
        agree = verifier_data.get("Agree", False)
        suggested_level = verifier_data.get("Suggested_Level", "C")
        justification = verifier_data.get("Justification", "")
    except:
        agree = False
        suggested_level = "C"
        justification = "Could not parse verifier response."

    # Decider Agent
    decider_prompt = f"""
    You are a neutral adjudicator evaluating the empathy level of a response post using the EPITOME rubric.

    Initial Classification: '{initial_level}'

    Seeker: {seeker_post}
    Response: {response_post}

    Verifier says:
    - Agrees with initial: {agree}
    - Suggested_Level: {suggested_level}
    - Justification: {justification}

    Make a final decision: A, B, or C.

    Respond in JSON:
    {{
        "Final_Level": "A", "B", or "C",
        "Rationale": "Why this final label best fits the rubric and context"
    }}
    """

    decider_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a neutral mental health expert finalizing the empathy classification."},
            {"role": "user", "content": decider_prompt}
        ]
    )

    try:
        decider_data = json.loads(clean_json(decider_completion.choices[0].message.content.strip()))
        final_level = decider_data.get("Final_Level", "C")
        final_rationale = decider_data.get("Rationale", "")
    except:
        final_level = "C"
        final_rationale = "Could not parse rationale."

    return final_level, final_rationale, justification, agree, suggested_level


In [ ]:
# def label_empathy(answer_file):
#     client = OpenAI(api_key=API_KEY)
#     answer_dict = {
#     "C": 0,
#     "B": 1,
#     "A": 2
#     }
#     ai_answers_with_debate = []
#     final_rationales = []
#     pro_view_args = []
#     anti_view_args = []
#     anti_view_suggested_levels = []

#     response_count = len(sampled_data)

#     for i in tqdm(range(response_count)):
#         chosen_messages = [sampled_data['seeker_post'].iloc[i], sampled_data['response_post'].iloc[i]]
#         prompt = f'''
#         You will be provided with a snippet of a conversation.
#         The first piece of text will be a support seeking post from an online user, and the second will be a response posted in response to the seeker's post.
#         Classify each query based on the level of empathy in the response post in the context of the seeker post.

#         We measure the level of empathy in terms of exploration, which is the pursuit of further information to better understand the seeker's situation.

#         Levels Of Empathy:
#         A. Strong Exploration: The response includes specific questions about the seeker's situation

#         B. Weak Exploration: The response includes general questions about the seeker's situation.

#         C. No Exploration: The response does not include any questions about the seeker's situation to further understand the seeker's situation.


#         Please classify the following responses based on the level of empathy in the response post in the context of the seeker post.
#         Seeker Post: {chosen_messages[0]}
#         Response Post: {chosen_messages[1]}

#         Answer with A, B, C and no justification for your answer.
#         Include no punctuation after your answer.
#         '''

#         completion = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "developer", "content": "You are a mental health professional."},
#                 {"role": "user", "content": prompt}
#             ]
#         )
#         initial_answer = completion.choices[0].message.content.strip()

#         # Debate and Decide
#         final_level, final_rationale, justification, agree, suggested_level = verifier_and_decide_empathy(
#             chosen_messages[0],
#             chosen_messages[1],
#             initial_answer,
#             prompt,
#             client
#         )

#         ai_answers_with_debate.append(answer_dict.get(final_level, 0))
#         final_rationales.append(final_rationale)
#         pro_view_args.append(justification)
#         anti_view_args.append(agree)
#         anti_view_suggested_levels.append(suggested_level)


#     # Save the results (you might want to save more than just the final level and rationale)
#     output_df = sampled_data.copy()
#     output_df['initial_ai_level'] = [answer_dict.get(lvl, 0) for lvl in [completion.choices[0].message.content.strip() for completion in [client.chat.completions.create(model="gpt-4o", messages=[{"role": "developer", "content": "You are a mental health professional."}, {"role": "user", "content": f'''You will be provided with a snippet of a conversation.\nThe first piece of text will be a support seeking post from an online user, and the second will be a response posted in response to the seeker's post.\nClassify each query based on the level of empathy in the response post in the context of the seeker post.\n\nWe measure the level of empathy in terms of exploration, which is the pursuit of further information to better understand the seeker's situation.\n\nLevels Of Empathy:\nA. Strong Exploration: The response includes specific questions about the seeker's situation\n\nB. Weak Exploration: The response includes general questions about the seeker's situation.\n\nC. No Exploration: The response does not include any questions about the seeker's situation to further understand the seeker's situation.\n\nPlease classify the following responses based on the level of empathy in the response post in the context of the seeker post.\nSeeker Post: {sampled_data['seeker_post'].iloc[i]}\nResponse Post: {sampled_data['response_post'].iloc[i]}\n\nAnswer with A, B, C and no justification for your answer.\nInclude no punctuation after your answer.\n'''}]) for i in range(response_count)]]] # This will be the initial level
#     output_df['debated_ai_level'] = ai_answers_with_debate
#     output_df['debated_ai_rationale'] = final_rationales
#     output_df['verifier_justification'] = pro_view_args
#     output_df['verifier_agreed'] = anti_view_args
#     output_df['verifier_suggested_level'] = [answer_dict.get(lvl, 0) for lvl in anti_view_suggested_levels]

#     output_df.to_csv(answer_file, index=False)


In [55]:
def label_empathy(answer_file):
    client = OpenAI(api_key=API_KEY)
    answer_dict = {
        "C": 0,
        "B": 1,
        "A": 2
    }
    
    # Determine the number of samples to process, e.g., 100 for testing or len(sampled_data) for all
    # For now, let's stick to the first 100 as in the previous example, or use len(sampled_data) for all
    # response_count = 100 
    response_count = len(sampled_data) # Or use a smaller number for testing, e.g., 100

    # Define column names for the CSV
    # Start with columns from sampled_data and add new AI-generated columns
    output_columns = list(sampled_data.columns) + [
        'initial_ai_level_char',      # e.g., "A", "B", "C"
        'initial_ai_level_num',       # e.g., 0, 1, 2
        'final_level_char',           # Final level from verifier_and_decide
        'final_level_num',
        'final_rationale',
        'verifier_justification',
        'verifier_agreed',            # true/false
        'verifier_suggested_level_char',
        'verifier_suggested_level_num'
    ]

    # Initialize CSV file with headers (this will overwrite the file if it exists)
    pd.DataFrame(columns=output_columns).to_csv(answer_file, index=False)
    print(f"Initialized {answer_file} with headers.")

    for i in tqdm(range(response_count)): # Loop through the specified number of samples
        chosen_messages = [sampled_data['seeker_post'].iloc[i], sampled_data['response_post'].iloc[i]]
        
        # Base prompt for initial classification
        prompt = f'''
        You will be provided with a snippet of a conversation.
        The first piece of text will be a support seeking post from an online user, and the second will be a response posted in response to the seeker's post.
        Classify each query based on the level of empathy in the response post in the context of the seeker post.

        We measure the level of empathy in terms of exploration, which is the pursuit of further information to better understand the seeker's situation.

        Levels Of Empathy:
        A. Strong Exploration: The response includes specific questions about the seeker's situation
        B. Weak Exploration: The response includes general questions about the seeker's situation.
        C. No Exploration: The response does not include any questions about the seeker's situation to further understand the seeker's situation.

        Please classify the following responses based on the level of empathy in the response post in the context of the seeker post.
        Seeker Post: {chosen_messages[0]}
        Response Post: {chosen_messages[1]}

        Answer with A, B, C and no justification for your answer.
        Include no punctuation after your answer.
        '''

        current_initial_answer_char = "C"  # Default value
        try:
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "developer", "content": "You are a mental health professional."},
                    {"role": "user", "content": prompt}
                ]
            )
            raw_initial_answer = completion.choices[0].message.content.strip()
            if raw_initial_answer in answer_dict: # Check if the raw answer is a valid key
                 current_initial_answer_char = raw_initial_answer
            else:
                print(f"Warning: Unexpected initial answer '{raw_initial_answer}' for item {i}. Defaulting to 'C'.")
        except Exception as e:
            print(f"Error getting initial AI level for item {i}: {e}. Defaulting to 'C'.")
            # current_initial_answer_char remains "C"

        # Verifier and Decide
        final_level_char = "C" # Default
        final_rationale_text = "Error in verifier/decide"
        verifier_justification_text = "Error in verifier/decide"
        verifier_agreed_bool = False # Default
        verifier_suggested_char = "C" # Default

        try:
            # Ensure verifier_and_decide_empathy is defined and returns these 5 values
            final_level_char, final_rationale_text, verifier_justification_text, verifier_agreed_bool, verifier_suggested_char = verifier_and_decide_empathy(
                chosen_messages[0],
                chosen_messages[1],
                current_initial_answer_char, # Pass the character A, B, or C
                prompt, # Pass the original prompt text
                client
            )
        except Exception as e:
            print(f"Error during verifier_and_decide_empathy for item {i}: {e}")
            # Defaults will be used

        # Prepare data for the current row
        row_data_dict = sampled_data.iloc[i].to_dict()

        # Add all AI generated data
        row_data_dict['initial_ai_level_char'] = current_initial_answer_char
        row_data_dict['initial_ai_level_num'] = answer_dict.get(current_initial_answer_char, 0)
        
        row_data_dict['final_level_char'] = final_level_char
        row_data_dict['final_level_num'] = answer_dict.get(final_level_char, 0)
        row_data_dict['final_rationale'] = final_rationale_text
        
        row_data_dict['verifier_justification'] = verifier_justification_text
        row_data_dict['verifier_agreed'] = verifier_agreed_bool
        
        row_data_dict['verifier_suggested_level_char'] = verifier_suggested_char
        row_data_dict['verifier_suggested_level_num'] = answer_dict.get(verifier_suggested_char, 0)

        # Convert the dictionary for the current row to a DataFrame
        current_row_df = pd.DataFrame([row_data_dict], columns=output_columns)
        
        # Append to CSV. mode='a' for append, header=False because headers are already written.
        current_row_df.to_csv(answer_file, mode='a', header=False, index=False)

    print(f"Processing complete. All results saved incrementally to {answer_file}")



In [56]:
answer_file = 'coded_sampled_data_gpt-4o_no_shot_verifier.csv'
label_empathy(answer_file)

Initialized coded_sampled_data_gpt-4o_no_shot_verifier.csv with headers.


100%|██████████| 1000/1000 [1:38:00<00:00,  5.88s/it]

Processing complete. All results saved incrementally to coded_sampled_data_gpt-4o_no_shot_verifier.csv


In [57]:
print("Metrics for Initial AI Response:")
calculate_f1_score(answer_file,'initial_ai_level_num', 'Initial AI Level')
calculate_cohens_kappa(answer_file, 'initial_ai_level_num', 'Initial AI Level')

print("\nMetrics for Final AI Response:")
calculate_f1_score(answer_file, 'final_level_num', 'Final AI Level')
calculate_cohens_kappa(answer_file, 'final_level_num', 'Final AI Level')

Metrics for Initial AI Response:
F1 Score (Initial AI Level): 0.64
Cohen's Kappa (Initial AI Level): 0.62
Weighted Cohen's Kappa (Initial AI Level): 0.74

Metrics for Final AI Response:
F1 Score (Final AI Level): 0.58
Cohen's Kappa (Final AI Level): 0.52
Weighted Cohen's Kappa (Final AI Level): 0.65


## Roles

In [ ]:
# def label_empathy(answer_file):
#     client = OpenAI(api_key=API_KEY)
#     answer_dict = {
#     "C": 0,
#     "B": 1,
#     "A": 2
#     }
#     initial_ai_levels = [] # To store the results from the initial prompt
#     ai_answers_with_roles = [] # To store the final roles-based majority vote
#     role_responses_list = [] # To store the individual responses from each role

#     response_count = len(sampled_data)

#     for i in tqdm(range(response_count)):
#         seeker_post = sampled_data['seeker_post'].iloc[i]
#         response_post = sampled_data['response_post'].iloc[i]
#         chosen_messages = [seeker_post, response_post] # Keep this for consistency with the original prompt format

#         # 1. Get the initial classification using the original prompt
#         initial_prompt_text = f'''
#         You will be provided with a snippet of a conversation.
#         The first piece of text will be a support seeking post from an online user, and the second will be a response posted in response to the seeker's post.
#         Classify each query based on the level of empathy in the response post in the context of the seeker post.

#         We measure the level of empathy in terms of exploration, which is the pursuit of further information to better understand the seeker's situation.

#         Levels Of Empathy:
#         A. Strong Exploration: The response includes specific questions about the seeker's situation

#         B. Weak Exploration: The response includes general questions about the seeker's situation.

#         C. No Exploration: The response does not include any questions about the seeker's situation to further understand the seeker's situation.


#         Please classify the following responses based on the level of empathy in the response post in the context of the seeker post.
#         Seeker Post: {chosen_messages[0]}
#         Response Post: {chosen_messages[1]}

#         Answer with A, B, C and no justification for your answer.
#         Include no punctuation after your answer.
#         '''
#         try:
#             initial_completion = client.chat.completions.create(
#                 model="gpt-4o",
#                 messages=[
#                     {"role": "user", "content": initial_prompt_text}
#                 ]
#             )
#             initial_answer = initial_completion.choices[0].message.content.strip()
#             # Extract the single letter classification
#             match = re.search(r'[ABC]', initial_answer)
#             if match:
#                 initial_level_letter = match.group(0)
#             else:
#                 initial_level_letter = "C" # Default if classification is not found
#         except Exception as e:
#             print(f"Error with API call for initial prompt item {i}: {e}")
#             initial_level_letter = "C" # Default on API error

#         initial_ai_levels.append(answer_dict.get(initial_level_letter, 0))


#         # 2. Define the empathy role-playing prompts, including the original prompt text
#         empathy_role_prompts = [
#             f"""You are a clinical mental health worker evaluating online support messages using the EPITOME framework.

# Here is the original task description you are working within:
# '''
# {initial_prompt_text}
# '''

# Context: A user has shared a personal post seeking emotional support. A peer has responded to it. Your role is to assess whether the peer made a genuine effort to understand the user's situation by asking meaningful questions.

# EPITOME Exploration Labels:
# A — Strong Exploration: Specific, detailed questions showing real curiosity about the user's circumstances.
# B — Weak Exploration: General or vague questions that show minimal probing.
# C — No Exploration: No follow-up questions or attempts to better understand the user's situation.

# Seeker's Post: {seeker_post}
# Response: {response_post}

# Classify the response as A, B, or C. Do not include explanation or punctuation.""",

#             f"""You are a trained peer support volunteer on an emotional support platform.

# Here is the original task description you are working within:
# '''
# {initial_prompt_text}
# '''

# Your goal is to reflect on whether the responder is actively trying to support the seeker by asking follow-up questions that show engagement and a desire to understand.

# Classification:
# - A: Strong Exploration — Very specific questions like “When did this start?” or “What happened after that?”
# - B: Weak Exploration — Vague or surface-level questions like “Are you okay?” or “What’s going on?”
# - C: No Exploration — The response is about the responder’s own experience or does not ask anything.

# Seeker: {seeker_post}
# Response: {response_post}

# Give only one letter: A, B, or C (no punctuation or commentary).""",

#             f"""You are an AI assistant trained to analyze empathetic language based on clinical labeling of peer responses.

# Here is the original task description you are working within:
# '''
# {initial_prompt_text}
# '''

# Task: Evaluate if the responder engaged in ‘exploration’—that is, asking questions to understand the seeker better.

# Labels:
# A — Strong Exploration: Specific, contextual questions showing thoughtful support.
# B — Weak Exploration: General or vague questions showing minimal probing.
# C — No Exploration: The response offers no questions to understand the seeker better.

# Seeker: {seeker_post}
# Response: {response_post}

# Provide only the letter (A, B, or C). No extra text.
# """
#         ]

#         # 3. Collect responses from each role
#         role_responses = []
#         for role_prompt in empathy_role_prompts:
#             try:
#                 completion = client.chat.completions.create(
#                     model="gpt-4o",
#                     messages=[
#                         {"role": "user", "content": role_prompt}
#                     ]
#                 )
#                 response = completion.choices[0].message.content.strip()
#                 # Extract the single letter classification
#                 match = re.search(r'[ABC]', response)
#                 if match:
#                     role_responses.append(match.group(0))
#                 else:
#                     role_responses.append("C") # Default to C if classification is not found
#             except Exception as e:
#                 print(f"Error with API call for item {i} and a role: {e}")
#                 role_responses.append("C") # Default to C on API error

#         role_responses_list.append(", ".join(role_responses))


#         # 4. Simple majority vote for the final roles-based decision
#         from collections import Counter
#         vote_counts = Counter(role_responses)
#         # Get the most common label. If there's a tie, choose one (e.g., the first one).
#         final_level_letter = vote_counts.most_common(1)[0][0]
#         final_level_numeric = answer_dict.get(final_level_letter, 0)

#         ai_answers_with_roles.append(final_level_numeric)


#     # 5. Save the results
#     output_df = sampled_data.copy()
#     output_df['initial_ai_level'] = initial_ai_levels # Store the initial classification
#     output_df['roles_based_level'] = ai_answers_with_roles # Store the majority vote result
#     output_df['role_individual_responses'] = role_responses_list # Store individual role responses


#     output_df.to_csv(answer_file, index=False)




In [58]:

def label_empathy(answer_file):
    client = OpenAI(api_key=API_KEY)
    answer_dict = {
        "C": 0,
        "B": 1,
        "A": 2
    }
    
    # Determine the number of samples to process
    response_count = len(sampled_data) # Or use a smaller number for testing, e.g., 100

    # Define column names for the CSV
    output_columns = list(sampled_data.columns) + [
        'initial_ai_level_char',       # e.g., "A", "B", "C"
        'initial_ai_level_num',        # e.g., 0, 1, 2
        'roles_based_level_char',      # Final level from roles
        'roles_based_level_num',
        'role_individual_responses_str' # Comma-separated string of role responses
    ]

    # Initialize CSV file with headers (this will overwrite the file if it exists)
    pd.DataFrame(columns=output_columns).to_csv(answer_file, index=False)
    print(f"Initialized {answer_file} with headers.")

    for i in tqdm(range(response_count)): # Loop through the specified number of samples
        seeker_post = sampled_data['seeker_post'].iloc[i]
        response_post = sampled_data['response_post'].iloc[i]
        chosen_messages = [seeker_post, response_post]

        # 1. Get the initial classification using the original prompt
        initial_prompt_text = f'''
        You will be provided with a snippet of a conversation.
        The first piece of text will be a support seeking post from an online user, and the second will be a response posted in response to the seeker's post.
        Classify each query based on the level of empathy in the response post in the context of the seeker post.

        We measure the level of empathy in terms of exploration, which is the pursuit of further information to better understand the seeker's situation.

        Levels Of Empathy:
        A. Strong Exploration: The response includes specific questions about the seeker's situation
        B. Weak Exploration: The response includes general questions about the seeker's situation.
        C. No Exploration: The response does not include any questions about the seeker's situation to further understand the seeker's situation.

        Please classify the following responses based on the level of empathy in the response post in the context of the seeker post.
        Seeker Post: {chosen_messages[0]}
        Response Post: {chosen_messages[1]}

        Answer with A, B, C and no justification for your answer.
        Include no punctuation after your answer.
        '''
        current_initial_level_char = "C" # Default
        try:
            initial_completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": initial_prompt_text}
                ]
            )
            initial_answer = initial_completion.choices[0].message.content.strip()
            match = re.search(r'[ABC]', initial_answer)
            if match:
                current_initial_level_char = match.group(0)
            else:
                print(f"Warning: Could not extract A, B, or C from initial answer '{initial_answer}' for item {i}. Defaulting to 'C'.")
        except Exception as e:
            print(f"Error with API call for initial prompt item {i}: {e}. Defaulting to 'C'.")
        
        current_initial_level_num = answer_dict.get(current_initial_level_char, 0)

        # 2. Define the empathy role-playing prompts
        empathy_role_prompts = [
            f"""You are a clinical mental health worker evaluating online support messages using the EPITOME framework.
Here is the original task description you are working within:
'''
{initial_prompt_text}
'''
Context: A user has shared a personal post seeking emotional support. A peer has responded to it. Your role is to assess whether the peer made a genuine effort to understand the user's situation by asking meaningful questions.
EPITOME Exploration Labels:
A — Strong Exploration: Specific, detailed questions showing real curiosity about the user's circumstances.
B — Weak Exploration: General or vague questions that show minimal probing.
C — No Exploration: No follow-up questions or attempts to better understand the user's situation.
Seeker's Post: {seeker_post}
Response: {response_post}
Classify the response as A, B, or C. Do not include explanation or punctuation.""",

            f"""You are a trained peer support volunteer on an emotional support platform.
Here is the original task description you are working within:
'''
{initial_prompt_text}
'''
Your goal is to reflect on whether the responder is actively trying to support the seeker by asking follow-up questions that show engagement and a desire to understand.
Classification:
- A: Strong Exploration — Very specific questions like “When did this start?” or “What happened after that?”
- B: Weak Exploration — Vague or surface-level questions like “Are you okay?” or “What’s going on?”
- C: No Exploration — The response is about the responder’s own experience or does not ask anything.
Seeker: {seeker_post}
Response: {response_post}
Give only one letter: A, B, or C (no punctuation or commentary).""",

            f"""You are an AI assistant trained to analyze empathetic language based on clinical labeling of peer responses.
Here is the original task description you are working within:
'''
{initial_prompt_text}
'''
Task: Evaluate if the responder engaged in ‘exploration’—that is, asking questions to understand the seeker better.
Labels:
A — Strong Exploration: Specific, contextual questions showing thoughtful support.
B — Weak Exploration: General or vague questions showing minimal probing.
C — No Exploration: The response offers no questions to understand the seeker better.
Seeker: {seeker_post}
Response: {response_post}
Provide only the letter (A, B, or C). No extra text."""
        ]

        # 3. Collect responses from each role
        current_role_responses_char = []
        for role_idx, role_prompt in enumerate(empathy_role_prompts):
            role_response_char = "C" # Default for this role
            try:
                completion = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "user", "content": role_prompt}
                    ]
                )
                response = completion.choices[0].message.content.strip()
                match = re.search(r'[ABC]', response)
                if match:
                    role_response_char = match.group(0)
                else:
                    print(f"Warning: Could not extract A, B, or C from role {role_idx} answer '{response}' for item {i}. Defaulting to 'C'.")
                current_role_responses_char.append(role_response_char)
            except Exception as e:
                print(f"Error with API call for item {i}, role {role_idx}: {e}. Defaulting to 'C'.")
                current_role_responses_char.append("C") # Default to C on API error
        
        current_role_responses_str = ", ".join(current_role_responses_char)

        # 4. Simple majority vote for the final roles-based decision
        from collections import Counter
        vote_counts = Counter(current_role_responses_char)
        
        final_roles_level_char = "C" # Default if no responses or tie with no clear winner
        if vote_counts:
            # Get the most common label. If there's a tie, Counter.most_common(1) picks one.
            # You might want more sophisticated tie-breaking if needed.
            final_roles_level_char = vote_counts.most_common(1)[0][0]
        
        final_roles_level_num = answer_dict.get(final_roles_level_char, 0)

        # Prepare data for the current row
        row_data_dict = sampled_data.iloc[i].to_dict()

        # Add all AI generated data
        row_data_dict['initial_ai_level_char'] = current_initial_level_char
        row_data_dict['initial_ai_level_num'] = current_initial_level_num
        
        row_data_dict['roles_based_level_char'] = final_roles_level_char
        row_data_dict['roles_based_level_num'] = final_roles_level_num
        row_data_dict['role_individual_responses_str'] = current_role_responses_str
        
        # Convert the dictionary for the current row to a DataFrame
        current_row_df = pd.DataFrame([row_data_dict], columns=output_columns)
        
        # Append to CSV. mode='a' for append, header=False because headers are already written.
        current_row_df.to_csv(answer_file, mode='a', header=False, index=False)

    print(f"Processing complete. All results saved incrementally to {answer_file}")


In [61]:
answer_file_roles = 'coded_sampled_data_gpt-4o_no_shot_roles.csv'
label_empathy(answer_file_roles)


Initialized coded_sampled_data_gpt-4o_no_shot_roles.csv with headers.


  2%|▏         | 22/1000 [00:49<36:44,  2.25s/it]

Error with API call for item 22, role 1: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-eMLxU7I2FH1eVZlBr5fna3f7 on tokens per min (TPM): Limit 30000, Used 29006, Requested 1237. Please try again in 486ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Defaulting to 'C'.


  2%|▏         | 23/1000 [00:56<1:01:46,  3.79s/it]

Error with API call for item 23, role 1: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-eMLxU7I2FH1eVZlBr5fna3f7 on tokens per min (TPM): Limit 30000, Used 29549, Requested 596. Please try again in 290ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Defaulting to 'C'.


...

Error with API call for initial prompt item 599: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-eMLxU7I2FH1eVZlBr5fna3f7 on tokens per min (TPM): Limit 30000, Used 29756, Requested 423. Please try again in 358ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Defaulting to 'C'.


 73%|███████▎  | 729/1000 [48:42<41:01,  9.08s/it]

Error with API call for initial prompt item 729: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-eMLxU7I2FH1eVZlBr5fna3f7 on tokens per min (TPM): Limit 30000, Used 29888, Requested 333. Please try again in 442ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Defaulting to 'C'.


100%|██████████| 1000/1000 [1:06:09<00:00,  3.97s/it]

Processing complete. All results saved incrementally to coded_sampled_data_gpt-4o_no_shot_roles.csv


In [63]:
print("Metrics for Initial AI Response:")
calculate_f1_score(answer_file_roles,'initial_ai_level_num', 'Initial AI Level')
calculate_cohens_kappa(answer_file_roles, 'initial_ai_level_num', 'Initial AI Level')

print("\nMetrics for Final AI Response:")
calculate_f1_score(answer_file_roles, 'roles_based_level_num', 'Final AI Level')
calculate_cohens_kappa(answer_file_roles, 'roles_based_level_num', 'Final AI Level')

Metrics for Initial AI Response:
F1 Score (Initial AI Level): 0.64
Cohen's Kappa (Initial AI Level): 0.64
Weighted Cohen's Kappa (Initial AI Level): 0.76

Metrics for Final AI Response:
F1 Score (Final AI Level): 0.70
Cohen's Kappa (Final AI Level): 0.70
Weighted Cohen's Kappa (Final AI Level): 0.80


## Debate and Decide Toxicity

In [ ]:
!pip install convokit

In [ ]:
from convokit import Corpus, download
corpus = Corpus(filename=download("conversations-gone-awry-corpus"))
# corpus = Corpus(filename=download("conversations-gone-awry-cmv-corpus"))

In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from openai import OpenAI
from convokit import Corpus, download


def clean_json(text):
    text = text.strip()
    if text.startswith("```json"):
        text = text[7:]
    if text.endswith("```"):
        text = text[:-3]
    return text


def get_first_two_utterances(convo):
    utts = convo.get_chronological_utterance_list()
    if len(utts) >= 2:
        return utts[0].text.strip(), utts[1].text.strip()
    return None, None


def build_cga_wiki_dataset(csv_path="cga_wiki_pairs.csv", jsonl_path="cga_wiki_pairs.jsonl"):
    corpus = Corpus(filename=download("conversations-gone-awry-corpus"))
    data = []

    for convo in corpus.iter_conversations():
        if convo.meta.get("conversation_has_personal_attack", False):
            pair_id = convo.meta.get("pair_id")
            if not pair_id or pair_id not in corpus.conversations:
                continue
            partner = corpus.get_conversation(pair_id)

            utt1, utt2 = get_first_two_utterances(convo)
            if utt1 and utt2:
                data.append({"conversation_id": convo.id, "text_pair": f"{utt1}\n{utt2}", "label": 1})

            utt1, utt2 = get_first_two_utterances(partner)
            if utt1 and utt2:
                data.append({"conversation_id": partner.id, "text_pair": f"{utt1}\n{utt2}", "label": 0})

    df = pd.DataFrame(data)
    df.to_csv(csv_path, index=False)
    df.to_json(jsonl_path, orient="records", lines=True)
    print(f"Saved {len(df)} examples to CSV and JSONL.")
    return df


def label_toxicity(cga_df, output_file, api_key, n_samples=10):
    client = OpenAI(api_key=api_key)

    initial_labels = []
    ai_labels = []
    final_rationales = []
    pro_view_args = []
    anti_view_args = []
    anti_view_suggestions = []

    for i in tqdm(range(n_samples)):
        thread_text = cga_df['text_pair'].iloc[i]

        prompt = f"""
        You will be provided with the first two messages from a Wikipedia talk page discussion.
        Based on these messages alone, predict whether the conversation is likely to turn toxic later on (i.e., contain a personal attack).

        Respond only with 'Yes' or 'No'. Do not include any explanation or punctuation.

        Messages:
        {thread_text}
        """

        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "developer", "content": "You are a Wikipedia moderator trained to spot early signs of toxicity."},
                {"role": "user", "content": prompt}
            ]
        )
        initial_label = completion.choices[0].message.content.strip()
        initial_numeric = 1 if initial_label == "Yes" else 0

        final_label, rationale, pro_arg, anti_arg, anti_suggest = debate_and_decide_toxicity(
            thread_text, initial_label, prompt, client
        )
        final_numeric = 1 if final_label == "Yes" else 0

        initial_labels.append(initial_numeric)
        ai_labels.append(final_numeric)
        final_rationales.append(rationale)
        pro_view_args.append(pro_arg)
        anti_view_args.append(anti_arg)
        anti_view_suggestions.append(anti_suggest)

    cga_df = cga_df.iloc[:n_samples].copy()
    cga_df['initial_ai_label'] = initial_labels
    cga_df['debated_ai_label'] = ai_labels
    cga_df['debated_ai_rationale'] = final_rationales
    cga_df['pro_view_argument'] = pro_view_args
    cga_df['anti_view_argument'] = anti_view_args
    cga_df['anti_view_suggested_label'] = anti_view_suggestions

    save_path = os.path.join(drive_folder, output_file)
    output_df.to_csv(save_path, index=False)
    return cga_df



def debate_and_decide_toxicity(thread_text, initial_label, original_prompt, client):
    # Pro View
    pro_prompt = f"""
    Given the following conversation:
    {thread_text}

    And an initial classification: {initial_label}

    Argue in support of the decision to classify this thread as '{initial_label}'.
    """
    pro_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Wikipedia moderator arguing in favor of the classification."},
            {"role": "user", "content": pro_prompt}
        ]
    )
    pro_argument = pro_completion.choices[0].message.content.strip()

    # Anti View
    anti_prompt = f"""
    Given the following conversation:
    {thread_text}

    And an initial classification: {initial_label}

    Argue why this classification might be incorrect. Suggest either 'Yes' or 'No' as a better label and explain why.

    Respond in JSON format:
    {{
        "Suggested_Label": "Yes" or "No",
        "Argument": "Your reasoning here"
    }}
    """
    anti_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Wikipedia moderator reviewing a peer's classification."},
            {"role": "user", "content": anti_prompt}
        ]
    )
    try:
        anti_data = json.loads(clean_json(anti_completion.choices[0].message.content.strip()))
        anti_suggested = anti_data.get("Suggested_Label", "No")
        anti_argument = anti_data.get("Argument", "")
    except:
        anti_suggested = "No"
        anti_argument = "Could not parse anti-view."

    # Decider
    decider_prompt = f"""
    Conversation:
    {thread_text}

    Initial Classification: {initial_label}
    Support Argument: {pro_argument}
    Opposition Argument: {anti_argument}
    Suggested Alternative: {anti_suggested}

    Decide on the best final label: 'Yes' or 'No'.

    Respond in JSON format:
    {{
        "Final_Label": "Yes" or "No",
        "Rationale": "Why this final label is best"
    }}
    """
    decider_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a neutral moderator deciding between two arguments."},
            {"role": "user", "content": decider_prompt}
        ]
    )
    try:
        decider_data = json.loads(clean_json(decider_completion.choices[0].message.content.strip()))
        final_label = decider_data.get("Final_Label", "No")
        rationale = decider_data.get("Rationale", "")
    except:
        final_label = "No"
        rationale = "Could not parse rationale."

    return final_label, rationale, pro_argument, anti_argument, anti_suggested





In [ ]:

csv_out = "debated_results_toxicity.csv"

# 1. Generate Data
df = build_cga_wiki_dataset()

# 2. Label Sample of 10
labeled_df = label_toxicity(df, csv_out, API_KEY, n_samples=1000)


In [ ]:
def calculate_f1_score_debated(ai_csv_path,  ground_truth_csv_path='sampled_data.csv'):
  ai_answers = pd.read_csv(ai_csv_path)['debated_ai_label']
  labels = pd.read_csv(ai_csv_path)['label']
  f1 = f1_score(labels, ai_answers, average='macro')
  print(f"F1 Score (Debated AI): {f1:.2f}")

def calculate_f1_score_initial(csv_path, ground_truth_csv_path='sampled_data.csv'):
  ai_answers = pd.read_csv(csv_path)['initial_ai_label']
  labels = pd.read_csv(csv_path)['label']
  f1 = f1_score(labels, ai_answers, average='macro')
  print(f"F1 Score (Initial AI): {f1:.2f}")

def calculate_f1_score_verifier(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['debated_ai_label']
    labels = pd.read_csv(csv_path)['label']
    f1 = f1_score(labels, ai_answers, average='macro')
    print(f"F1 Score (Verifier Suggestion): {f1:.2f}")

def calculate_cohens_kappa_debated(csv1, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv1)
    ratings1 = df1['debated_ai_label']
    ratings2 = df1['label']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Debated AI): {normal_kappa:.2f}\nWeighted Cohen's Kappa (Debated AI): {weighted_kappa:.2f}")

def calculate_cohens_kappa_initial(csv_path, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv_path)
    ratings1 = df1['initial_ai_label']
    ratings2 = df1['label']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Initial AI): {normal_kappa:.2f}\nWeighted Cohen's Kappa (Initial AI): {weighted_kappa:.2f}")

def calculate_cohens_kappa_verifier(csv_path, ground_truth_csv_path='sampled_data.csv'):
    df1 = pd.read_csv(csv_path)
    ratings1 = df1['debated_ai_label']
    ratings2 = df1['label']
    normal_kappa = cohen_kappa_score(ratings1, ratings2)
    weighted_kappa = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    print(f"Cohen's Kappa (Verifier Suggestion): {normal_kappa:.2f}")
    print(f"Weighted Cohen's Kappa (Verifier Suggestion): {weighted_kappa:.2f}")

In [ ]:
# 3. Evaluate
print("Metrics for Initial AI Response:")
calculate_f1_score_initial(csv_out)
calculate_cohens_kappa_initial(csv_out)

print("\nMetrics for Debated AI Response:")
calculate_f1_score_debated(csv_out)
calculate_cohens_kappa_debated(csv_out)

In [ ]:
def label_toxicity(cga_df, output_file, api_key, n_samples=10):
    client = OpenAI(api_key=api_key)

    initial_labels = []
    ai_labels = []
    final_rationales = []
    pro_view_args = []
    anti_view_args = []
    anti_view_suggestions = []

    for i in tqdm(range(n_samples)):
        thread_text = cga_df['text_pair'].iloc[i]

        prompt = f"""
        You will be provided with the first two messages from a Wikipedia talk page discussion.
        Based on these messages alone, predict whether the conversation is likely to turn toxic later on (i.e., contain a personal attack).

        Respond only with 'Yes' or 'No'. Do not include any explanation or punctuation.

        Messages:
        {thread_text}
        """

        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "developer", "content": "You are a Wikipedia moderator trained to spot early signs of toxicity."},
                {"role": "user", "content": prompt}
            ]
        )
        initial_label = completion.choices[0].message.content.strip()
        initial_numeric = 1 if initial_label == "Yes" else 0

        final_label, rationale, verifier_justification, agree, suggested_label = verifier_and_decide_toxicity(
            thread_text, initial_label, prompt, client
        )
        final_numeric = 1 if final_label == "Yes" else 0

        initial_labels.append(initial_numeric)
        ai_labels.append(final_numeric)
        final_rationales.append(rationale)
        pro_view_args.append(verifier_justification)
        anti_view_args.append(agree)
        anti_view_suggestions.append(suggested_label)

    cga_df = cga_df.iloc[:n_samples].copy()
    cga_df['initial_ai_label'] = initial_labels
    cga_df['debated_ai_label'] = ai_labels
    cga_df['debated_ai_rationale'] = final_rationales
    cga_df['verifier_justification'] = verifier_justification
    cga_df['verifier_agreed'] = agree
    cga_df['verifier_suggested_label'] = suggested_label

    save_path = os.path.join(drive_folder, output_file)
    output_df.to_csv(save_path, index=False)
    return cga_df

def verifier_and_decide_toxicity(thread_text, initial_label, original_prompt, client):
    # Verifier Agent
    verifier_prompt = f"""
    You are a verifier evaluating whether the initial classification is justified.

    Initial Classification: '{initial_label}'
    Conversation:
    {thread_text}

    Determine if the initial label is valid based on the conversation context. If you disagree, suggest a better label.

    Respond in JSON:
    {{
        "Agree": true or false,
        "Suggested_Label": "Yes" or "No",
        "Justification": "..."
    }}
    """

    verifier_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Wikipedia moderator acting as a verifier of classification decisions."},
            {"role": "user", "content": verifier_prompt}
        ]
    )

    verifier_raw = verifier_completion.choices[0].message.content.strip()
    try:
        verifier_data = json.loads(clean_json(verifier_raw))
        agree = verifier_data.get("Agree", False)
        suggested_label = verifier_data.get("Suggested_Label", "No")
        justification = verifier_data.get("Justification", "")
    except:
        agree = False
        suggested_label = "No"
        justification = "Could not parse verifier response."

    # Decider Agent
    decider_prompt = f"""
    You are a neutral evaluator using the task rubric to make a final judgment.

    Initial Classification: '{initial_label}'
    Conversation:
    {thread_text}

    Verifier response:
    Agree = {agree}
    Suggested_Label = {suggested_label}
    Justification = {justification}

    Decide whether to stick with the initial label or use the verifier's alternative. Justify your answer.

    Respond in JSON:
    {{
        "Final_Label": "Yes" or "No",
        "Rationale": "Brief reason for your decision"
    }}
    """

    decider_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a neutral moderator deciding whether to change the classification."},
            {"role": "user", "content": decider_prompt}
        ]
    )

    try:
        decider_data = json.loads(clean_json(decider_completion.choices[0].message.content.strip()))
        final_label = decider_data.get("Final_Label", "No")
        rationale = decider_data.get("Rationale", "")
    except:
        final_label = "No"
        rationale = "Could not parse rationale."

    return final_label, rationale, justification, agree, suggested_label


In [ ]:
csv_out = "verified_results_toxicity.csv"

# 1. Generate Data
df = build_cga_wiki_dataset()

# 2. Label Sample of 10
labeled_df = label_toxicity(df, csv_out, API_KEY, n_samples=1000)



In [ ]:
# 3. Evaluate
print("Metrics for Initial AI Response:")
calculate_f1_score_initial(csv_out)
calculate_cohens_kappa_initial(csv_out)

print("\nMetrics for Verifier AI Response:")
calculate_f1_score_verifier(csv_out)
calculate_cohens_kappa_verifier(csv_out)

roles


In [ ]:
def social_scientist_roles_toxicity(thread_text, initial_label, client):
    # Social Scientist 1: Empirical analyst
    analyst_prompt = f"""
You are a social scientist studying online disinhibition and the early signs of toxicity in Wikipedia discussions.

Conversation:
{thread_text}

The current prediction is: '{initial_label}' — this means the thread *is* (Yes) or *is not* (No) likely to become toxic.

Do you agree with this prediction based on sociolinguistic cues (like escalating tone, sarcasm, or confrontation)? If not, suggest a corrected label.

Respond in JSON:
{{
  "Agree": true or false,
  "Suggested_Label": "Yes" or "No",
  "Justification": "Explain based on observed early cues"
}}
"""

    analyst_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a sociolinguist trained in online behavior analysis."},
            {"role": "user", "content": analyst_prompt}
        ]
    )

    try:
        data = json.loads(clean_json(analyst_response.choices[0].message.content.strip()))
        agree = data.get("Agree", False)
        suggested_label = data.get("Suggested_Label", "No")
        justification = data.get("Justification", "")
    except:
        agree = False
        suggested_label = "No"
        justification = "Parsing error: unable to extract rationale."

    # Final Decision: Group Judgment
    judge_prompt = f"""
You are summarizing a decision made by a panel of social scientists reviewing the following discussion:

{thread_text}

The original model classified the thread as: '{initial_label}'.

Panel member response:
- Agreed with model? {agree}
- Suggested_Label: {suggested_label}
- Rationale: {justification}

Based on this input, choose the best final label: 'Yes' (likely to become toxic later) or 'No' (unlikely to become toxic).

Respond in JSON:
{{
  "Final_Label": "Yes" or "No",
  "Rationale": "Justify the final label based on social science reasoning"
}}
"""

    judge_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are synthesizing feedback from a panel of researchers on toxicity prediction."},
            {"role": "user", "content": judge_prompt}
        ]
    )

    try:
        final_data = json.loads(clean_json(judge_response.choices[0].message.content.strip()))
        final_label = final_data.get("Final_Label", "No")
        rationale = final_data.get("Rationale", "")
    except:
        final_label = "No"
        rationale = "Parsing error in final decision."

    return final_label, rationale, justification, agree, suggested_label

def label_toxicity(cga_df, output_file, api_key, n_samples=10):
    client = OpenAI(api_key=api_key)

    initial_labels = []
    ai_labels = []
    final_rationales = []
    pro_view_args = []
    anti_view_args = []
    anti_view_suggestions = []

    for i in tqdm(range(n_samples)):
        thread_text = cga_df['text_pair'].iloc[i]

        prompt = f"""
        You will be provided with the first two messages from a Wikipedia talk page discussion.
        Based on these messages alone, predict whether the conversation is likely to turn toxic later on (i.e., contain a personal attack).

        Respond only with 'Yes' or 'No'. Do not include any explanation or punctuation.

        Messages:
        {thread_text}
        """

        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "developer", "content": "You are a Wikipedia moderator trained to spot early signs of toxicity."},
                {"role": "user", "content": prompt}
            ]
        )
        initial_label = completion.choices[0].message.content.strip()
        initial_numeric = 1 if initial_label == "Yes" else 0

        final_label, rationale, justification, agree, suggested_label = social_scientist_roles_toxicity(
            thread_text, initial_label, client
        )
        final_numeric = 1 if final_label == "Yes" else 0

        initial_labels.append(initial_numeric)
        ai_labels.append(final_numeric)
        final_rationales.append(rationale)
        pro_view_args.append(justification)
        anti_view_args.append(agree)
        anti_view_suggestions.append(suggested_label)

    cga_df = cga_df.iloc[:n_samples].copy()
    cga_df['initial_ai_label'] = initial_labels
    cga_df['debated_ai_label'] = ai_labels
    cga_df['debated_ai_rationale'] = final_rationales
    cga_df['panel_justification'] = pro_view_args
    cga_df['panel_agreement'] = anti_view_args
    cga_df['panel_suggested_label'] = anti_view_suggestions


    cga_df.to_csv(output_file, index=False)
    return cga_df


In [ ]:
def calculate_f1_score_roles(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['debated_ai_label']
    true_labels = pd.read_csv(csv_path)['label']
    f1 = f1_score(true_labels, ai_answers)
    print(f"F1 Score (Roles AI): {f1:.2f}")

def calculate_cohens_kappa_roles(csv_path, ground_truth_csv_path='sampled_data.csv'):
    ai_answers = pd.read_csv(csv_path)['debated_ai_label']
    true_labels = pd.read_csv(csv_path)['label']
    kappa = cohen_kappa_score(true_labels, ai_answers)
    print(f"Cohen's Kappa (Roles AI): {kappa:.2f}")
    weighted_kappa = cohen_kappa_score(true_labels, ai_answers, weights='quadratic')
    print(f"Weighted Cohen's Kappa (Verifier Suggestion): {weighted_kappa:.2f}")


In [ ]:
csv_out = "roles_results_toxicity.csv"

# 1. Generate Data
df = build_cga_wiki_dataset()

# 2. Label Sample of 10
labeled_df = label_toxicity(df, csv_out, API_KEY, n_samples=1000)

In [ ]:
# 3. Evaluate
print("Metrics for Initial AI Response:")
calculate_f1_score_initial(csv_out)
calculate_cohens_kappa_initial(csv_out)

print("\nMetrics for Roles AI Response:")
calculate_f1_score_roles(csv_out)
calculate_cohens_kappa_roles(csv_out)

### Model Evaluation

## Part 4: Visualization

### Model Performance Visualization